# 1. Introdução

Nesta aula temos como um exemplo uma base de Dados que contém:

    1) O nome de cada aluno
    2) Suas notas em diversas avaliações
    3) Se ele foi adimitido em um processo seletivo a partir de suas notas (ou não).

Sendo "ser admitido" = 1 e "não ser admitido" = 0. Este é o item que chamamos de variável "dependente" e as demais são as "independentes".

In [1]:
# Importando principais pacotes e a base
import numpy as np
import pandas as pd
baseDeDados = pd.read_csv('admission.csv', delimiter=';')
baseDeDados

,Name,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Approval
0,Lucas,337,118,4,4.5,4.5,9.65,1,1
1,Ana,324,107,4,4.0,4.5,8.87,1,1
2,Jose,316,104,3,3.0,3.5,8.00,1,1
3,Carlos,322,110,3,3.5,2.5,8.67,1,1
4,Zileide,314,103,2,2.0,3.0,8.21,0,0
5,Joana,330,115,5,4.5,3.0,9.34,1,1
6,Davi,321,109,3,3.0,4.0,8.20,1,1
7,Daniel,308,101,2,3.0,4.0,7.90,0,0
8,Marcelo,302,102,1,2.0,1.5,8.00,0,0


Vamos separar as variáveis independentes em uma base X e as variáveis dependentes em uma base y.

In [2]:
X = baseDeDados.iloc[:,:-1].values # Da primeira coluna parando antes da última
y = baseDeDados.iloc[:,-1].values # Apenas a última
X

array([['Lucas', 337, 118, 4, 4.5, 4.5, 9.65, 1],
       ['Ana', 324, 107, 4, 4.0, 4.5, 8.87, 1],
       ['Jose', 316, 104, 3, 3.0, 3.5, 8.0, 1],
       ['Carlos', 322, 110, 3, 3.5, 2.5, 8.67, 1],
       ['Zileide', 314, 103, 2, 2.0, 3.0, 8.21, 0],
       ['Joana', 330, 115, 5, 4.5, 3.0, 9.34, 1],
       ['Davi', 321, 109, 3, 3.0, 4.0, 8.2, 1],
       ['Daniel', 308, 101, 2, 3.0, 4.0, 7.9, 0],
       ['Marcelo', 302, 102, 1, 2.0, 1.5, 8.0, 0]], dtype=object)

# 2. Limpeza (revisão da aula passada)

Agora iremos fazer a limpeza de dados da base X (nesse exemplo específico não há dados faltantes, mas faremos mesmo assim).

Importante: quando fazemos a limpeza dessa forma, os valores categóricos (o nome, nesse caso, é automaticamente excluído).

In [3]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median') #Configurando o "imputador de valores"
imputer = imputer.fit(X[:,1:]) # Ajusta o imputador aos dados forneceidos (pula a coluna dos nomes)
X = imputer.transform(X[:,1:]) # Vai aplicar o imputador ajustado as colunas necessárias
X

array([[337.  , 118.  ,   4.  ,   4.5 ,   4.5 ,   9.65,   1.  ],
       [324.  , 107.  ,   4.  ,   4.  ,   4.5 ,   8.87,   1.  ],
       [316.  , 104.  ,   3.  ,   3.  ,   3.5 ,   8.  ,   1.  ],
       [322.  , 110.  ,   3.  ,   3.5 ,   2.5 ,   8.67,   1.  ],
       [314.  , 103.  ,   2.  ,   2.  ,   3.  ,   8.21,   0.  ],
       [330.  , 115.  ,   5.  ,   4.5 ,   3.  ,   9.34,   1.  ],
       [321.  , 109.  ,   3.  ,   3.  ,   4.  ,   8.2 ,   1.  ],
       [308.  , 101.  ,   2.  ,   3.  ,   4.  ,   7.9 ,   0.  ],
       [302.  , 102.  ,   1.  ,   2.  ,   1.5 ,   8.  ,   0.  ]])

# 3. LabelEncoder

Vamos ver o que é possível fazer com os nomes dos alunos para incluí-los na análise. Vamos começar utilizando mais uma ferramenta da biblioteca scikit-learn:

**LabelEncoder** é uma classe da biblioteca scikit-learn que é usada para codificar rótulos categóricos em valores numéricos. Em outras palavras, ela converte rótulos de texto em números.

Quando você tem dados categóricos (por exemplo, "maçã", "laranja", "banana"), muitos algoritmos de aprendizado de máquina requerem que esses rótulos sejam numéricos para que possam ser processados adequadamente.

Então vamos começar recuperando os nomes que perdemos na limpeza:

In [4]:
baseDeDados.iloc[:,0].values

array(['Lucas', 'Ana', 'Jose', 'Carlos', 'Zileide', 'Joana', 'Davi',
       'Daniel', 'Marcelo'], dtype=object)

Agora vamos fazer utilizar o LabelEnconder nesses nomes:

In [5]:
from sklearn.preprocessing import LabelEncoder # Importamos a classe da biblioteca
# Utilizamos um objeto desta classe para codificar a primeira coluna e atribuímos a ela mesma
NomesEncodados = LabelEncoder().fit_transform(baseDeDados.iloc[:,0].values) 
# Imprimindo
NomesEncodados

array([6, 0, 5, 1, 8, 4, 3, 2, 7])

Muito legal! Cada valor categórico foi codifiado como valores numéricos.

Porém essa codificação, da forma que foi feita, pode não ser boa para a nossa análise. Afinal, cada nome foi assignado um valor randômico e esses valores podem afetar o treinamento do modelo de forma equivocada. O modelo pode interpretar que ter valores mais altos ou baixos na classe de nome aumenta ou abaixa as chances um aluno ser aprovado). Vamos aprender outra técnica que pode ser mais útil nesses casos.

# 4. **One-Hot Encoding** ou **Dummy Encoding**

Vamos conhecer o **One-Hot Encoding** ou **Dummy Encoding**. Esta técnica cria uma nova coluna para cada categoria única presente na variável categórica. Cada nova coluna é binária e indica se a observação pertence a essa categoria ou não. Também é chamado de "dummy encoding".

In [6]:
# Com get_dummies vamos criar uma coluna para cada variável categórica
D = pd.get_dummies(baseDeDados.iloc[:,0].values)
D

,Ana,Carlos,Daniel,Davi,Joana,Jose,Lucas,Marcelo,Zileide
0,False,False,False,False,False,False,True,False,False
1,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False
3,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True
5,False,False,False,False,True,False,False,False,False
6,False,False,False,True,False,False,False,False,False
7,False,False,True,False,False,False,False,False,False
8,False,False,False,False,False,False,False,True,False


Por fim, vamos unir essas colunas às colunas de X que já passaram pelo processo de limpeza.

In [7]:
# Insere as colunas ajustadas pelo imputador 
X = np.insert(X, #Array ao qual irei adicionar valores
              0, #Em qual índice os valores serão adicionados
              D.values, #Os valores que serão adicionados
              axis =1) #Se o índice se refere a linhas ou colunas.

X

array([[  0.  ,   1.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  , 337.  , 118.  ,   4.  ,   4.5 ,   4.5 ,   9.65,   1.  ],
       [  0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  , 324.  , 107.  ,   4.  ,   4.  ,   4.5 ,   8.87,   1.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,
          0.  , 316.  , 104.  ,   3.  ,   3.  ,   3.5 ,   8.  ,   1.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  , 322.  , 110.  ,   3.  ,   3.5 ,   2.5 ,   8.67,   1.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,
          0.  , 314.  , 103.  ,   2.  ,   2.  ,   3.  ,   8.21,   0.  ],
       [  0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  , 330.  , 115.  ,   5.  ,   4.5 ,   3.  ,   9.34,   1.  ],
       [  1.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  , 321.  , 109.  ,   3.  ,   3.  ,   4.  ,   

Obs: Esse contexto de "nomes de alunos" e em que cada valor é único não seria tão útil para um modelo de machine learning neste caso, mas pense que poderia ser uma outra categoria como "escola em que estudou" e houvessem valores repetidos para observarmos se escolas específicas estariam associadas positivamente a passar em concursos (mais do que "se o nome do aluno for Lucas ele tem mais chance de passar em um concurso").

# 5. Train_test_split

Agora que temos a base tratada, vamos introduzir um assunto muito importante em Machine Learning: divisão de base de treino e teste.

De forma simplificada: Em problemas de classificação de Machine Learning, a máquina primeiro estuda uma base de treino e vê o que é mais ou menos associado a variável explicativa. A partir disso ela treina um modelo. 

No caso deste exercício, por exemplo a máquina vai observar as informações das variáveis independentes (X) da base de treino (nome do aluno, suas notas, desempenhos) e observará na base dependente (y) se aquele aluno passou ou não no processo seletivo. A partir dessa observação ele irá **treinar** um modelo.

Mas como sabemos se o modelo ficou bom ou não? Testando. Utilizando a base de teste que separamos, imputando os variáveis independentes (X) no modelo e vendo quais são as previsões da variável dependente (y) que ele faz. A partir das previsões do modelo utilizando a base de testes, vamos comparar o com y teste real (que separamos lá no início) e vendo o quanto o modelo foi capaz de acertar suas previsões.

Para fazer isso vamos utiliazr a função "train_test_split" do módulo "model_selection" do pacote "sklearn". 

In [8]:
from sklearn.model_selection import train_test_split
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2) #O test_size pode mudar se você quiser outro tamanho.

Assim nós dividimos 80% da nossa base para treino e 20% para teste. Obs: A ordem antes das variáveis antes do "=" tem que ser a acima para o processo de divisão dar certo. 

Cada base ficará assim (porém, geralmente, com um conjunto de dados muito maior):

In [9]:
pd.DataFrame(XTrain)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,324.0,107.0,4.0,4.0,4.5,8.87,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,314.0,103.0,2.0,2.0,3.0,8.21,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,302.0,102.0,1.0,2.0,1.5,8.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,322.0,110.0,3.0,3.5,2.5,8.67,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,308.0,101.0,2.0,3.0,4.0,7.90,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,316.0,104.0,3.0,3.0,3.5,8.00,1.0
6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,337.0,118.0,4.0,4.5,4.5,9.65,1.0


In [10]:
pd.DataFrame(yTrain)

,0
0,1
1,0
2,0
3,1
4,0
5,1
6,1


In [11]:
pd.DataFrame(XTest)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,321.0,109.0,3.0,3.0,4.0,8.20,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,330.0,115.0,5.0,4.5,3.0,9.34,1.0


In [12]:
pd.DataFrame(yTest)

,0
0,1
1,1
